In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
os.getcwd()

'/Users/austinclime/vs_code_projects/taq_data_pull/python_code'

In [3]:
os.listdir()

['testing.ipynb', '__pycache__', 'wrds', 'nbbo_query.py', 'db_connection.py']

In [7]:
import wrds

In [9]:
db = wrds.Connection(wrds_username='aclime2')
#db.create_pgpass_file()#aclime2

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [10]:
df = db.raw_sql("""
    SELECT
        SUBSTR(time_m::VARCHAR,1,5) AS time_m,
        best_bid,
        ROUND(hr_avg_prc,2) AS hr_avg_pr,
        best_bid - ROUND(hr_avg_prc,2) AS diff
    FROM
        (
            SELECT
                *,
                RANK() OVER (PARTITION BY sym_root, date, EXTRACT (HOUR FROM time_m), EXTRACT (MINUTE FROM time_m) ORDER BY time_m, time_m_nano) AS minute_rank,
                AVG(best_bid) OVER (PARTITION BY sym_root, date, EXTRACT (HOUR FROM time_m)) AS hr_avg_prc
            FROM
                taqm_2022.complete_nbbo_2022
            WHERE
                sym_root = 'AAPL' AND
                date = '20220930' 
        ) a
    WHERE
        minute_rank = 1
    ORDER BY
        1
     """)
df.head()

,time_m,best_bid,hr_avg_pr,diff
0,04:00,141.85,143.21,-1.36
1,04:01,143.38,143.21,0.17
2,04:02,143.24,143.21,0.03
3,04:03,143.31,143.21,0.10
4,04:04,143.24,143.21,0.03


In [1]:
from db_connection import get_db_connection
db=get_db_connection()

ModuleNotFoundError: No module named 'wrds'

In [4]:
from nbbo_query import make_req

In [7]:
req=make_req('AAPL',2022,9,30)
df = db.raw_sql(req)

In [8]:
df

,time_m,best_bid,best_ask
0,09:30,141.23,141.34
1,09:31,141.67,141.70
2,09:32,142.02,142.04
3,09:33,142.20,142.21
4,09:34,142.47,142.48
...,...,...,...
385,15:55,138.68,138.70
386,15:56,138.42,138.43
387,15:57,138.32,138.33
388,15:58,138.26,138.27
